In [1]:
import numpy as np

In [2]:
class Variable:
    def __init__(self, array):
        self._array = np.asarray(array)
        self._op = None
        self._inputs = []
        self.grad = None

    def __str__(self):
        return str(self._array)
    
    def __repr__(self):
        return 'Variable(' + str(self) + ')'

    def __add__(self, other):
        return AddOp.apply(self, other)

    def __sub__(self, other):
        return SubOp.apply(self, other)

    def __mul__(self, other):
        return MulOp.apply(self, other)
    
    def mean(self):
        return MeanOp.apply(self)
    
    def backward(self, grad=None):
        if grad is None:
            grad = np.array(1)
        if self._op is None:
            if self.grad is None:
                self.grad = np.zeros_like(self._array)
            self.grad += grad
            return
        grads = self._op.backward(self, grad)
        for v, g in zip(self._inputs, grads):
            v.backward(g)

In [3]:
class Function:
    @classmethod
    def apply(cls, *inputs):
        v = cls.forward(*inputs)
        v._op = cls
        v._inputs = inputs
        return v

class AddOp(Function):
    @staticmethod
    def forward(a, b):
        return Variable(a._array + b._array)
    @staticmethod
    def backward(y, grad):
        return grad, grad
    
class MulOp(Function):
    @staticmethod
    def forward(a, b):
        return Variable(a._array * b._array)
    @staticmethod
    def backward(y, grad):
        return grad * y._inputs[1]._array, grad * y._inputs[0]._array   

class SubOp(Function):
    @staticmethod
    def forward(a, b):
        return Variable(a._array - b._array)
    @staticmethod
    def backward(y, grad):
        return grad, -grad

class MeanOp(Function):
    @staticmethod
    def forward(a):
        return Variable(a._array.mean())
    @staticmethod
    def backward(y, grad):
        return (grad / y._inputs[0]._array.size,)

In [4]:
x = Variable([0.3, 0.4])
W = Variable([0.4, 0.5])
b = Variable([0.5, -0.3])
y = Variable([1., 2.])
l = ((W * x + b - y) * (W * x + b - y)).mean()
l

Variable(2.2772)

In [5]:
l.backward()

In [6]:
W.grad

array([-0.114, -0.84 ])

In [7]:
b.grad

array([-0.38, -2.1 ])